# Handwriting Neural Network Demonstration

This notebook demonstrates a simple neural network model for handwritten digit classification.

## Import Required Libraries

Import the necessary libraries including TensorFlow, matplotlib, and numpy.

In [ ]:
# Import the required libraries.
from onnxmltools import convert_keras
from onnxmltools.utils import save_model

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

## Load and Preprocess Data

Load the MNIST dataset and preprocess the data for training.

In [ ]:
# Load the dataset.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Determine 10% of the training and test sets.
train_size = int(0.1 * len(x_train))
test_size = int(0.1 * len(x_test))

# Randomly select 10% of the data.
np.random.seed(42)  # For reproducibility
train_indices = np.random.choice(len(x_train), train_size, replace=False)
test_indices = np.random.choice(len(x_test), test_size, replace=False)

x_train_small = x_train[train_indices]
y_train_small = y_train[train_indices]
x_test_small = x_test[test_indices]
y_test_small = y_test[test_indices]

# Preprocess the data.
x_train_small = x_train_small.astype("float32") / 255
x_test_small = x_test_small.astype("float32") / 255

# Convert labels to categorical one-hot encoding.
y_train_small = to_categorical(y_train_small, 10)
y_test_small = to_categorical(y_test_small, 10)

## Build Neural Network Model

Create a sequential neural network model for digit classification.

In [ ]:
# Build the model.
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(10, activation="softmax")
])

# Save the model structure as an image.
plot_model(
    model, to_file="./lecture07/models/mnist_model.png",
    show_shapes=True, show_layer_names=True
)

## Compile and Train Model

Compile the model with appropriate optimizer and loss function, then train it.

In [ ]:
# Compile the model.
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

# Train the model.
model.fit(
    x_train_small, y_train_small,
    epochs=5, batch_size=512, validation_split=0.2
)

## Evaluate Model Performance

Evaluate the trained model on the test dataset.

In [ ]:
# Evaluate the model.
test_loss, test_acc = model.evaluate(x_test_small, y_test_small)
print(f"Test accuracy: {test_acc}.")

## Export Model to ONNX Format

Convert the trained model to ONNX format for deployment.

In [ ]:
# Convert the model to ONNX format.
model.output_names = [ "output" ]
input_signature = [
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name="digit")
]
onnx_model = convert_keras(model, target_opset=13)

# Save the ONNX model to a file.
save_model(onnx_model, "./lecture07/models/mnist_model.onnx")

## Visualize Predictions

Show sample test images with actual and predicted labels.

In [ ]:
# Show some test data and the actual and predicted classes in a 2x3 plot.
NUM_SAMPLES = 6
indices = np.random.choice(len(x_test_small), NUM_SAMPLES, replace=False)
sample_images = x_test_small[indices]
sample_labels = y_test_small[indices]
predictions = model.predict(sample_images)

fig, axes = plt.subplots(2, 3, figsize=(10, 7))
axes = axes.flatten()

for i in range(NUM_SAMPLES):
    axes[i].imshow(sample_images[i], cmap="gray")
    axes[i].set_title(
        f"Actual: {np.argmax(sample_labels[i])}, "
        f"Predicted: {np.argmax(predictions[i])}"
    )
    axes[i].axis("off")

plt.tight_layout()
plt.show()